In [ ]:
import pandas as pd
import numpy as np

In [ ]:
cred = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/scenario_credences/scenario.credences.csv', index_col = 0)

# Create the probability weighted totals for affectability AND raw datasets
affectable = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/affectability_data/all_countries_emit_weighted_by_affectability.csv', index_col = 0)
raw = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/master_file/affectable_emissions_MASTER.csv', index_col = 0)

# example files for plot testing
ex_good = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/scenario_credences/scenario.credencesEXTREME_GOOD.csv', index_col = 0)
ex_bad = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/scenario_credences/scenario.credencesEXTREME_BAD.csv', index_col = 0)

In [ ]:
def make_expectation_over_data(data, credence, area):
    # make a column in the data data which will match the creedences data, merge on this column
    clean_rcp = data['RCP'].str.replace('.', '')
    data['cred_scen_match'] = data['SSP'] + '-' + clean_rcp
    data['cred_scen_match'] = data['cred_scen_match'].str.replace('BL', 'Baseline')

    scen_and_prob = data.merge(credence, left_on = 'cred_scen_match', right_on = 'scenario', how = 'inner').fillna(0)
    sort_scen_prob = scen_and_prob.sort_values(by = 'country').reset_index().drop(columns = 'index')

    # make a copy of the merged data in order to weight it
    scen_and_prob_weighted = sort_scen_prob.copy()

    # Multiply every emissions value in the dataset by it's affectability, converting the data to affectable emissions

    # make active cols which should be weighted separate variable, matters b/c prob vars shouldn't be included
    # in final output or get weighted by the following calculation
    prob_cols_to_ignore = ['country', 'region', 'SSP', 'RCP', 'cred_scen_match','scenario','credence','posterior','p']

    # multiply each row of emissions (but not p itself) in the merged dataset of global emissions 
    # by its data's p value
    for col in scen_and_prob_weighted:
        if col not in prob_cols_to_ignore:
            scen_and_prob_weighted[col] = scen_and_prob_weighted[col] * scen_and_prob_weighted['p']

    expectation = scen_and_prob_weighted.groupby(by = area).agg(np.nansum).reset_index()
    return(expectation)

In [ ]:
ex_goodp = make_expectation_over_data(raw, ex_good, 'region')
ex_badp = make_expectation_over_data(raw, ex_bad, 'region')

ex_goodp.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/p_weighted_ex_good.csv')
ex_badp.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/p_weighted_ex_bad.csv')

In [ ]:
expectation_affectable = make_expectation_over_data(affectable, cred, 'country') 
expectation_raw = make_expectation_over_data(raw, cred, 'country')
expectation_affectableRegion = make_expectation_over_data(affectable, cred, 'region') 
expectation_rawRegion = make_expectation_over_data(raw, cred, 'region')

expectation_affectable.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/p_weighted_scen_all_countries_regionsAFFECTABLE.csv')
expectation_raw.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/p_weighted_scen_all_countries_regionsRAW.csv')

expectation_affectableRegion.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/p_weighted_scen_all_regions_AFFECTABLE.csv')
expectation_rawRegion.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/p_weighted_scen_all_regions_RAW.csv')